In [ ]:
import os
import librosa
import numpy as np
import pandas as pd

base_dir = r'MyDrive'
categories = ['ASD', 'NON-ASD']

def extract_audio_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    rms = librosa.feature.rms(y=y)
    features_df = pd.DataFrame([[np.mean(spectral_centroid), np.mean(spectral_rolloff),
                                 np.mean(zero_crossing_rate), np.mean(rms)]],
                                columns=['spectral_centroid', 'spectral_rolloff', 'zero_crossing_rate', 'rms'])
    return features_df

for category in categories:
    test_folder_path = os.path.join(base_dir, category)
    output_folder_path = os.path.join(base_dir, 'audiofeatureextracted',
                                      'spectral_centroid_spectral_rolloff_zero_crossing_rate_rms/')

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for filename in os.listdir(test_folder_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(test_folder_path, filename)
            features_df = extract_audio_features(file_path)

            counter = 1
            output_csv_path = os.path.join(output_folder_path, f"extracted_features_{category}_{counter}.csv")
            while os.path.exists(output_csv_path):
                counter += 1
                output_csv_path = os.path.join(output_folder_path, f"extracted_features_{category}_{counter}.csv")

            features_df.to_csv(output_csv_path, index=False)
            print(f"Features for '{filename}' saved to {output_csv_path}")


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_validate
import numpy as np

feature_combinations = [
    "spectral_centroid",
    "spectral_rolloff",
    "zero_crossing_rate",
    "rms",
    "spectral_centroid_zero_crossing_rate",
    "spectral_centroid_spectral_rolloff",
    "spectral_centroid_rms",
    "spectral_rolloff_zero_crossing_rate",
    "spectral_rolloff_rms",
    "zero_crossing_rate_rms",
    "spectral_centroid_spectral_rolloff_zero_crossing_rate",
    "spectral_centroid_spectral_rolloff_rms",
    "spectral_centroid_zero_crossing_rate_rms",
    "spectral_rolloff_zero_crossing_rate_rms"
]

def load_data(base_dir, categories):
    data = []
    labels = []

    for category in categories:
        category_dir = os.path.join(base_dir, category)
        label = 1 if category == 'ASD' else 0  

        for filename in os.listdir(category_dir):
            if filename.endswith('.csv'):
                file_path = os.path.join(category_dir, filename)

                try:
                    df = pd.read_csv(file_path, encoding='utf-8', on_bad_lines='skip')
                except UnicodeDecodeError:
                    df = pd.read_csv(file_path, encoding='ISO-8859-1', on_bad_lines='skip')

                features = df.values.flatten()
                data.append(features)
                labels.append(label)

    data = pd.DataFrame(data)
    labels = pd.Series(labels)

    return data, labels

def evaluate_model(base_dir, feature_combination):
    categories = ['ASD', 'NON-ASD']
    data, labels = load_data(base_dir, categories)

    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    svm_model = SVC(kernel='linear', random_state=42)

    scoring = {
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, pos_label=1),
        'recall': make_scorer(recall_score, pos_label=1),
        'f1': make_scorer(f1_score, pos_label=1)
    }

    cv_results = cross_validate(svm_model, data_scaled, labels, cv=cv, scoring=scoring)

    accuracy_mean = np.mean(cv_results['test_accuracy']) * 100
    accuracy_std = np.std(cv_results['test_accuracy'])

    precision_mean = np.mean(cv_results['test_precision'])
    precision_std = np.std(cv_results['test_precision'])

    recall_mean = np.mean(cv_results['test_recall'])
    recall_std = np.std(cv_results['test_recall'])

    f1_mean = np.mean(cv_results['test_f1'])
    f1_std = np.std(cv_results['test_f1'])

    print(f"Feature Combination: {feature_combination.replace('_', ' ')}")
    print(f"Cross-Validation Accuracy: {accuracy_mean:.2f}% ± {accuracy_std:.2f}")
    print(f"Cross-Validation Precision (ASD): {precision_mean:.2f} ± {precision_std:.2f}")
    print(f"Cross-Validation Recall (ASD): {recall_mean:.2f} ± {recall_std:.2f}")
    print(f"Cross-Validation F1 Score (ASD): {f1_mean:.2f} ± {f1_std:.2f}")
    print('-' * 80)

for feature_combination in feature_combinations:
    base_dir = f'drive/MyDrive/audiofeatureextracted/{feature_combination}'
    evaluate_model(base_dir, feature_combination)
